In [1]:
from PIL import Image
import numpy as np
import os 
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
import torch
import sys
sys.path.append('../codes/')
from network import CNN_with_decoder,CNN2
from losses import  loss
from optimizer import create_optimizer
%load_ext autoreload
%autoreload 2

In [2]:
train_path = '../data/train/'
all_positive_train = os.listdir('../data/train/1/')
all_negative_train = os.listdir('../data/train/0/')


In [7]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
])
dataset = datasets.ImageFolder(
    root=os.path.join('../data/', 'train'),
    transform=transform_train)

fp=open('../logs/output.log','a+')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN2().to(device)
# 定义代价函数
criterion = torch.nn.CrossEntropyLoss()

# 定义优化器
optimizer = torch.optim.Adam(model.parameters())

epochs = 10

In [4]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(dataset, test_size=0.3, random_state=42)


In [5]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=128, shuffle=False)

In [8]:
from engine import train
model_trained, best_model, train_los, train_acc, val_los, val_acc = train(
    model=model, 
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    max_epoch=epochs,
    disp_freq=100)

Epoch [0][10]	 Batch [0][589]	 Training Loss 0.7328	 Accuracy 0.5547	 Time(Iter) 0.5458
Epoch [0][10]	 Batch [100][589]	 Training Loss 0.8433	 Accuracy 0.4698	 Time(Iter) 0.3510
Epoch [0][10]	 Batch [200][589]	 Training Loss 0.8401	 Accuracy 0.4731	 Time(Iter) 0.3622
Epoch [0][10]	 Batch [300][589]	 Training Loss 0.8412	 Accuracy 0.4720	 Time(Iter) 0.3629
Epoch [0][10]	 Batch [400][589]	 Training Loss 0.8430	 Accuracy 0.4702	 Time(Iter) 0.3795
Epoch [0][10]	 Batch [500][589]	 Training Loss 0.8417	 Accuracy 0.4715	 Time(Iter) 0.3739
Epoch [0]	 Average training loss 0.8432	 Average training accuracy 0.4700
Epoch [0]	 Average validation loss 0.8456	 Average validation accuracy 0.4676
Best ckpt 0.4676177537017189
Epoch [1][10]	 Batch [0][589]	 Training Loss 0.8836	 Accuracy 0.4297	 Time(Iter) 0.3568
Epoch [1][10]	 Batch [100][589]	 Training Loss 0.8447	 Accuracy 0.4685	 Time(Iter) 0.3557
Epoch [1][10]	 Batch [200][589]	 Training Loss 0.8449	 Accuracy 0.4684	 Time(Iter) 0.3587
Epoch [1][10]

In [28]:
import cv2 as cv
img = cv.imread('../data/train/1/000001.SZ-20100526.png')

# sharpen_op = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
# sharpen_image = cv.filter2D(img, cv.CV_32F, sharpen_op)
# sharpen_image = cv.convertScaleAbs(sharpen_image)
# sharpen_image[sharpen_image>0]=255
# plt.imshow(sharpen_image)

In [3]:
# backtest_frame_path = 'D:/QuantFrame'
backtest_frame_path = '/home/qzliu/QuantFrame'

import pandas as pd
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.rcParams['font.size'] = '20' # 设置字体大小 
sns.set()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import sys 
sys.path.append(backtest_frame_path)
import numba

from datetime import datetime
from tqdm import tqdm
from multiprocessing import Pool,cpu_count
try: 
    import my_config as config
except:
    import config
    print('Failed to import module my_config.py, use default config file.')

data_path = config.data_path

In [4]:
from Data import TDayProcessor,TushareDownloader,StockData
from Factors import Factor, TSFeatureEngineer, NumbaFuncs, TimingFactor,ScreenFactor
from StockBackTest import BackTest, EventStudy
from AutoAlpha import AlphaTreePopulation, ExpressionTree,Node


from AutoAlpha.Population import calculate_pop_tree_value,evaluate_pop_tree
from AutoAlpha.AlphaTree import split_string_with_balanced_parentheses

%load_ext autoreload
%autoreload 2
%matplotlib inline

date_col = 'TradingDay'
secu_col = 'SecuCode'
key_col = [date_col,secu_col]

In [5]:
DB = StockData(data_path,start_dt="2009-01-01",update=False)

HDFReader: invalid sort_by columns.


In [6]:
data_folder = '../data_base_500'

all_files = os.listdir(os.path.join(data_folder,'train','0')) + \
            os.listdir(os.path.join(data_folder,'train','1')) + \
            os.listdir(os.path.join(data_folder,'test','0')) + \
            os.listdir(os.path.join(data_folder,'test','1'))

def parse_info(file_name):
    secu_code = file_name.split('.')[0]
    trading_day = file_name.split('-')[1].rstrip('.png')
    trading_day = trading_day[0:4] + '-' + trading_day[4:6] + '-' + trading_day[6:]
    return secu_code, trading_day

secu_list, trading_day_list = zip(*[parse_info(file_name) for file_name in all_files])

In [7]:
tmp_secu = list(set(secu_list))

In [8]:
for secu in tmp_secu:
    if not (secu.startswith('0') or secu.startswith('6')):
        print(secu)

In [16]:
df = DB.daily_df_qfq.query('SecuCode in @tmp_secu and TradingDay < "2015-01-01"')
df.reset_index(inplace=True,drop=True)
df.head(3)

,SecuCode,TradingDay,open,high,low,close,TurnoverVolume,TurnoverValue,is_new,SWF,SWS,SWT,adj_factor,vwap
0,000006,2009-01-05,NaN,NaN,NaN,NaN,5183639.0,26503711.6,False,农林牧渔,种植业,种子,NaN,NaN
1,000021,2009-01-05,NaN,NaN,NaN,NaN,2908118.0,12423674.7,False,农林牧渔,种植业,种子,NaN,NaN
2,000028,2009-01-05,NaN,NaN,NaN,NaN,1016654.0,17171821.4,False,农林牧渔,种植业,种子,NaN,NaN


In [17]:
df['open_60D'] = df.groupby('SecuCode')['open'].rolling(60,min_periods=30).apply(lambda x: x.values[0]).droplevel(0) 
df['low_60D'] = df.groupby('SecuCode')['low'].rolling(60,min_periods=30).min().droplevel(0) 
df['high_60D'] = df.groupby('SecuCode')['high'].rolling(60,min_periods=30).max().droplevel(0) 
df['open_60D_pct'] = (df['open_60D']/df['close'] - 1) * 100
df['low_60D_pct'] = (df['low_60D']/df['close'] - 1) * 100
df['high_60D_pct'] = (df['high_60D']/df['close'] - 1) * 100

In [18]:
df['secu'] = df['SecuCode'].map(lambda x: x + ('.SZ' if x.startswith('0') else '.SH'))
df['date'] = df['TradingDay'].map(lambda x: datetime.strftime(x,'%Y%m%d'))
df['idx'] = df['secu'] + '-' + df['date'] + '.png'

In [19]:

df = df[['idx','SecuCode','TradingDay','open_60D_pct','low_60D_pct','high_60D_pct']].query('TradingDay >= "2010-01-01"').set_index('idx').dropna()

In [20]:
df.query('TradingDay < "2015-01-01"').to_hdf('../data_base_500/price.h5',key='x')

In [21]:
df.query('TradingDay >= "2013-01-01"').to_hdf('../data_base_test_acc/price.h5',key='x')

In [22]:
df[['open_60D_pct', 'low_60D_pct','high_60D_pct']].describe()

,open_60D_pct,low_60D_pct,high_60D_pct
count,532894.000000,532894.000000,532894.000000
mean,0.946422,-14.334541,17.786865
std,18.742542,10.301312,13.775298
min,-82.596846,-83.550711,0.000000
25%,-11.574742,-20.347619,7.496464
50%,0.317993,-12.511993,14.666667
75%,12.533034,-6.182213,24.650350
max,231.267874,0.000000,296.186845
